In [1]:
#===========================================================
# Base code from https://www.kaggle.com/phoenix9032/pytorch-bert-plain
#===========================================================
import os
import sys
import gc
import time
import glob
import multiprocessing
import re
from urllib.parse import urlparse
from tqdm import tqdm
from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
from contextlib import contextmanager
from functools import partial

import numpy as np
import pandas as pd
import scipy as sp
from scipy.stats import spearmanr
import math
from math import floor, ceil
import random

#from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit, MultilabelStratifiedKFold
from sklearn.model_selection import GroupKFold
import category_encoders as ce
import re
from urllib.parse import urlparse

import torch
import torch.nn.functional as F
from torch import nn
from torch.utils import data
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
import transformers
from transformers import (
    BertTokenizer, BertModel, BertForSequenceClassification, BertConfig,
    WEIGHTS_NAME, CONFIG_NAME, AdamW, get_linear_schedule_with_warmup,
    get_cosine_schedule_with_warmup,
)
from transformers.modeling_bert import BertPreTrainedModel 


#===========================================================
# Utils
#===========================================================
def get_logger(filename='log'):
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

logger = get_logger()


@contextmanager
def timer(name):
    t0 = time.time()
    logger.info(f'[{name}] start')
    yield
    logger.info(f'[{name}] done in {time.time() - t0:.0f} s')


def seed_everything(seed=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

#===========================================================
# Config
#===========================================================
class PipeLineConfig:
    def __init__(self, lr, warmup, accum_steps, epochs, seed, expname, 
                 head_tail, head, freeze, question_weight, answer_weight, fold, train, cv, test):
        self.lr = lr
        self.warmup = warmup
        self.accum_steps = accum_steps
        self.epochs = epochs
        self.seed = seed
        self.expname = expname
        self.head_tail = head_tail
        self.head = head
        self.freeze = freeze
        self.question_weight = question_weight
        self.answer_weight = answer_weight
        self.fold = fold
        self.train = train
        self.cv = cv
        self.test = test

config = PipeLineConfig(lr=1e-4, warmup=0.1, accum_steps=1, epochs=6,
                        seed=42, expname='uncased_6', head_tail=True, head=0.3, freeze=False,
                        question_weight=0., answer_weight=0., fold=5, train=False, cv=False, test=True)

DEBUG = False
ID = 'qa_id'
target_cols = ['question_asker_intent_understanding', 'question_body_critical', 
               'question_conversational', 'question_expect_short_answer', 
               'question_fact_seeking', 'question_has_commonly_accepted_answer', 
               'question_interestingness_others', 'question_interestingness_self', 
               'question_multi_intent', 'question_not_really_a_question', 
               'question_opinion_seeking', 'question_type_choice',
               'question_type_compare', 'question_type_consequence',
               'question_type_definition', 'question_type_entity', 
               'question_type_instructions', 'question_type_procedure', 
               'question_type_reason_explanation', 'question_type_spelling', 
               'question_well_written', 'answer_helpful',
               'answer_level_of_information', 'answer_plausible', 
               'answer_relevance', 'answer_satisfaction', 
               'answer_type_instructions', 'answer_type_procedure', 
               'answer_type_reason_explanation', 'answer_well_written']
NUM_FOLDS = config.fold
ROOT = '../input/google-quest-challenge/'
#ROOT = '../input/'
SEED = config.seed
seed_everything(SEED)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
MODEL_DIR = '../input/googlequestchallenge-weights1/'
#MODEL_DIR = './'
COMBINE_INPUT = False
T_MAX_LEN = 30
Q_MAX_LEN = 479 # 382
A_MAX_LEN = 510 # 254 
MAX_SEQUENCE_LENGTH = T_MAX_LEN + Q_MAX_LEN + A_MAX_LEN + 4
q_max_sequence_length = T_MAX_LEN + Q_MAX_LEN + 3
a_max_sequence_length = A_MAX_LEN + 2

#===========================================================
# Model
#===========================================================
class OptimizedRounder5(object):
    def __init__(self):
        self.coef = [0.3333, 0.5, 0.6667, 1.]

    def _loss(self, X, y):
        X_p = np.digitize(X, self.coef)
        ll = spearmanr(y, X_p).correlation
        return -ll

    def fit(self, X: np.ndarray, y: np.ndarray):
        golden1 = 0.618
        golden2 = 1 - golden1
        ab_start = [(0., 0.3333), (0.3333, 0.5), (0.5, 0.6667), (0.6667, 1.)]
        for _ in range(100):
            search = iter(range(4))
            for idx in search:
                # golden section search
                a, b = ab_start[idx]
                # calc losses
                self.coef[idx] = a
                la = self._loss(X, y)
                self.coef[idx] = b
                lb = self._loss(X, y)
                for it in range(4):
                    # choose value
                    if la > lb:
                        a = b - (b - a) * golden1
                        self.coef[idx] = a
                        la = self._loss(X, y)
                    else:
                        b = b - (b - a) * golden2
                        self.coef[idx] = b
                        lb = self._loss(X, y)

    def predict(self, X, coef):
        X_p = np.digitize(X, coef)
        return X_p
    
    def coefficients(self):
        return self.coef


def _get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens)>max_seq_length:
        print(f'len(tokens): {len(tokens)}')
        print(f'max_seq_length: {max_seq_length}')
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))


def _get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    
    if len(tokens) > max_seq_length:
        raise IndexError("Token length more than max seq length!")
        
    segments = []
    first_sep = True
    current_segment_id = 0
    
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            if first_sep:
                first_sep = False 
            else:
                current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))


def _get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids


def _trim_input(tokenizer, title, question, answer, max_sequence_length, t_max_len, q_max_len, a_max_len):
    
    # 350+128+30 = 508 +4 = 512
    
    t = tokenizer.tokenize(title)
    q = tokenizer.tokenize(question)
    a = tokenizer.tokenize(answer)
    
    t_len = len(t)
    q_len = len(q)
    a_len = len(a)

    if (t_len+q_len+a_len+4) > max_sequence_length:
        
        if t_max_len > t_len:
            t_new_len = t_len
            a_max_len = a_max_len + floor((t_max_len - t_len)/2)
            q_max_len = q_max_len + ceil((t_max_len - t_len)/2)
        else:
            t_new_len = t_max_len
      
        if a_max_len > a_len:
            a_new_len = a_len 
            q_new_len = q_max_len + (a_max_len - a_len)
        elif q_max_len > q_len:
            a_new_len = a_max_len + (q_max_len - q_len)
            q_new_len = q_len
        else:
            a_new_len = a_max_len
            q_new_len = q_max_len
            
            
        if t_new_len+a_new_len+q_new_len+4 != max_sequence_length:
            raise ValueError("New sequence length should be %d, but is %d"%(max_sequence_length, (t_new_len + a_new_len + q_new_len + 4)))
        # Head+Tail method 
        q_len_head = round(q_new_len * config.head)
        q_len_tail = -1 * (q_new_len - q_len_head)
        a_len_head = round(a_new_len * config.head)
        a_len_tail = -1 * (a_new_len - a_len_head)
        t_len_head = round(t_new_len * config.head)
        t_len_tail = -1 * (t_new_len - t_len_head)  
        #t = t[:t_new_len]
        if config.head_tail :
            q = q[:q_len_head]+q[q_len_tail:]
            a = a[:a_len_head]+a[a_len_tail:]
            #t = t[:t_len_head]+t[t_len_tail:]
            t = t[:t_new_len]
        else:
            # No Head+Tail , usual processing
            q = q[:q_new_len]
            a = a[:a_new_len]
            t = t[:t_new_len]
    
    return t, q, a


def q_trim_input(tokenizer, title, question, q_max_sequence_length, t_max_len, q_max_len):

    t = tokenizer.tokenize(title)
    q = tokenizer.tokenize(question)

    t_len = len(t)
    q_len = len(q)

    if (t_len+q_len+3) > q_max_sequence_length:

        if t_max_len > t_len:
            t_new_len = t_len
            q_max_len = q_max_len + (t_max_len - t_len)
        else:
            t_new_len = t_max_len

        if q_max_len > q_len:
            q_new_len = q_len
            t_new_len = t_max_len + (q_max_len - q_len)
        else:
            q_new_len = q_max_len

        # Head+Tail method
        q_len_head = round(q_new_len * config.head)
        q_len_tail = -1 * (q_new_len - q_len_head)
        t_len_head = round(t_new_len * config.head)
        t_len_tail = -1 * (t_new_len - t_len_head)
        #t = t[:t_new_len]
        if config.head_tail :
            q = q[:q_len_head]+q[q_len_tail:]
            t = t[:t_len_head]+t[t_len_tail:]
            #t = t[:t_new_len]
        else:
            # No Head+Tail , usual processing
            q = q[:q_new_len]
            t = t[:t_new_len]

    return t, q


def a_trim_input(tokenizer, answer, a_max_sequence_length, a_max_len):

    a = tokenizer.tokenize(answer)

    a_len = len(a)

    if (a_len+2) > a_max_sequence_length:

        a_new_len = a_max_len

        # Head+Tail method
        a_len_head = round(a_new_len * config.head)
        a_len_tail = -1 * (a_new_len - a_len_head)
        if config.head_tail :
            a = a[:a_len_head]+a[a_len_tail:]
        else:
            # No Head+Tail , usual processing
            a = a[:a_new_len]

    return a


def _convert_to_bert_inputs(title, question, answer, tokenizer, max_sequence_length):
    """Converts tokenized input to ids, masks and segments for BERT"""
    if COMBINE_INPUT:
        stoken = ["[CLS]"] + title + ["[QBODY]"] + question + ["[ANS]"] + answer + ["[SEP]"]
        #stoken = ["[CLS]"] + title + ["[SEP]"] + question + ["[SEP]"] + answer + ["[SEP]"]
        #stoken = ["[CLS]"] + title  + question  + answer + ["[SEP]"]
    
        input_ids = _get_ids(stoken, tokenizer, max_sequence_length)
        input_masks = _get_masks(stoken, max_sequence_length)
        input_segments = _get_segments(stoken, max_sequence_length)

        return [input_ids, input_masks, input_segments]
    else:
        q_token = ["[CLS]"] + title + ["[SEP]"] + question + ["[SEP"]
        q_input_ids = _get_ids(q_token, tokenizer, T_MAX_LEN+Q_MAX_LEN+3)
        q_input_masks = _get_masks(q_token, T_MAX_LEN+Q_MAX_LEN+3)
        q_input_segments = _get_segments(q_token, T_MAX_LEN+Q_MAX_LEN+3)
        
        a_token = ["[CLS]"] + answer + ["[SEP]"]
        a_input_ids = _get_ids(a_token, tokenizer, A_MAX_LEN+2)
        a_input_masks = _get_masks(a_token, A_MAX_LEN+2)
        a_input_segments = _get_segments(a_token, A_MAX_LEN+2)

        return [q_input_ids, q_input_masks, q_input_segments, a_input_ids, a_input_masks, a_input_segments]


def compute_input_arays(df, columns, tokenizer, max_sequence_length, num_features, cat_features, 
                        t_max_len=T_MAX_LEN, q_max_len=Q_MAX_LEN, a_max_len=A_MAX_LEN):
    if COMBINE_INPUT:
        input_ids, input_masks, input_segments = [], [], []
        for _, instance in df[columns].iterrows():
            t, q, a = instance.question_title, instance.question_body, instance.answer
            t, q, a = _trim_input(tokenizer, t, q, a, max_sequence_length, t_max_len, q_max_len, a_max_len)
            ids, masks, segments = _convert_to_bert_inputs(t, q, a, tokenizer, max_sequence_length)
            input_ids.append(ids)
            input_masks.append(masks)
            input_segments.append(segments)
        return [
                torch.from_numpy(np.asarray(input_ids, dtype=np.int32)).long(), 
                torch.from_numpy(np.asarray(input_masks, dtype=np.int32)).long(),
                torch.from_numpy(np.asarray(input_segments, dtype=np.int32)).long(),
                torch.from_numpy(np.asarray(num_features, dtype=np.float32)).float(),
                torch.from_numpy(np.asarray(cat_features, dtype=np.int32)).long(),
                ]
    else:
        q_input_ids, q_input_masks, q_input_segments = [], [], []
        a_input_ids, a_input_masks, a_input_segments = [], [], []
        for _, instance in df[columns].iterrows():
            t, q, a = instance.question_title, instance.question_body, instance.answer
            t, q = q_trim_input(tokenizer, t, q, q_max_sequence_length, t_max_len, q_max_len)
            a = a_trim_input(tokenizer, a, a_max_sequence_length, a_max_len)
            q_ids, q_masks, q_segments, a_ids, a_masks, a_segments = _convert_to_bert_inputs(t, q, a, tokenizer, max_sequence_length)
            q_input_ids.append(q_ids)
            q_input_masks.append(q_masks)
            q_input_segments.append(q_segments)
            a_input_ids.append(a_ids)
            a_input_masks.append(a_masks)
            a_input_segments.append(a_segments)
        return [
                torch.from_numpy(np.asarray(q_input_ids, dtype=np.int32)).long(),
                torch.from_numpy(np.asarray(q_input_masks, dtype=np.int32)).long(),
                torch.from_numpy(np.asarray(q_input_segments, dtype=np.int32)).long(),
                torch.from_numpy(np.asarray(a_input_ids, dtype=np.int32)).long(),
                torch.from_numpy(np.asarray(a_input_masks, dtype=np.int32)).long(),
                torch.from_numpy(np.asarray(a_input_segments, dtype=np.int32)).long(),
                torch.from_numpy(np.asarray(num_features, dtype=np.float32)).float(),
                torch.from_numpy(np.asarray(cat_features, dtype=np.int32)).long(),
                ]


def compute_output_arrays(df, columns):
    return np.asarray(df[columns])


if COMBINE_INPUT:

    class QuestDataset(torch.utils.data.Dataset):
        def __init__(self, inputs, lengths, labels = None):

            self.inputs = inputs
            if labels is not None:
                self.labels = labels
            else:
                self.labels = None
            self.lengths = lengths

        def __getitem__(self, idx):

            input_ids       = self.inputs[0][idx]
            input_masks     = self.inputs[1][idx]
            input_segments  = self.inputs[2][idx]
            num_features    = self.inputs[3][idx]
            cat_features    = self.inputs[4][idx]
            lengths         = self.lengths[idx]
            if self.labels is not None: # targets
                labels = self.labels[idx]
                return input_ids, input_masks, input_segments, num_features, cat_features, labels, lengths
            return input_ids, input_masks, input_segments, num_features, cat_features, lengths

        def __len__(self):
            return len(self.inputs[0])


    class CustomBert(BertPreTrainedModel):

        def __init__(self, config, cat_dims):
            super(CustomBert, self).__init__(config)
            self.num_labels = config.num_labels
            self.bert = BertModel(config)
            self.embeddings = nn.ModuleList([
                nn.Embedding(x, y) for x, y in cat_dims
            ])
            self.emb_drop = nn.Dropout(0.2)
            n_emb_out = sum([y for x, y in cat_dims])
            self.dropout = nn.Dropout(0.2)
            self.classifier_final = nn.Linear(config.hidden_size+n_emb_out+4, self.config.num_labels)  # num_features=4

            self.init_weights()

        def forward(
            self,
            input_ids=None,
            attention_mask=None,
            token_type_ids=None,
            num_features=None,
            cat_features=None,
            position_ids=None,
            head_mask=None,
            inputs_embeds=None,
            labels=None,
        ):

            outputs = self.bert(
                input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids,
                position_ids=position_ids,
                head_mask=head_mask,
                inputs_embeds=inputs_embeds,
            )

            pooled_output = outputs[1]
            pooled_output = self.dropout(pooled_output)

            emb = [
                emb_layer(cat_features[:, j]) for j, emb_layer in enumerate(self.embeddings)
            ]
            emb = self.emb_drop(torch.cat(emb, 1))

            pooled_output = torch.cat([pooled_output, num_features, emb], 1)
            logits = self.classifier_final(pooled_output)

            outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here
            if labels is not None:
                if self.num_labels == 1:
                    #  We are doing regression
                    loss_fct = MSELoss()
                    loss = loss_fct(logits.view(-1), labels.view(-1))
                else:
                    loss_fct = CrossEntropyLoss()
                    loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
                outputs = (loss,) + outputs

            return outputs  # (loss), logits, (hidden_states), (attentions)

else:

    class QuestDataset(torch.utils.data.Dataset):
        def __init__(self, inputs, lengths, labels = None):

            self.inputs = inputs
            if labels is not None:
                self.labels = labels
            else:
                self.labels = None
            self.lengths = lengths

        def __getitem__(self, idx):

            q_input_ids       = self.inputs[0][idx]
            q_input_masks     = self.inputs[1][idx]
            q_input_segments  = self.inputs[2][idx]
            a_input_ids       = self.inputs[3][idx]
            a_input_masks     = self.inputs[4][idx]
            a_input_segments  = self.inputs[5][idx]
            num_features    = self.inputs[6][idx]
            cat_features    = self.inputs[7][idx]
            lengths         = self.lengths[idx]
            if self.labels is not None: # targets
                labels = self.labels[idx]
                return q_input_ids, q_input_masks, q_input_segments, a_input_ids, a_input_masks, a_input_segments, num_features, cat_features, labels, lengths
            return q_input_ids, q_input_masks, q_input_segments, a_input_ids, a_input_masks, a_input_segments, num_features, cat_features, lengths

        def __len__(self):
            return len(self.inputs[0])


    class CustomBert(BertPreTrainedModel):

        def __init__(self, config, cat_dims):
            super(CustomBert, self).__init__(config)
            self.num_labels = config.num_labels
            self.bert = BertModel(config)
            self.embeddings = nn.ModuleList([
                nn.Embedding(x, y) for x, y in cat_dims
            ])
            self.emb_drop = nn.Dropout(0.2)
            n_emb_out = sum([y for x, y in cat_dims])
            self.q_dropout = nn.Dropout(0.2)
            self.a_dropout = nn.Dropout(0.2)
            self.classifier_final = nn.Linear(config.hidden_size*2+n_emb_out+4, self.config.num_labels)  # num_features=4

            self.init_weights()

        def forward(
            self,
            q_input_ids=None,
            q_attention_mask=None,
            q_token_type_ids=None,
            a_input_ids=None,
            a_attention_mask=None,
            a_token_type_ids=None,
            num_features=None,
            cat_features=None,
            position_ids=None,
            head_mask=None,
            inputs_embeds=None,
            labels=None,
        ):

            q_outputs = self.bert(
                q_input_ids,
                attention_mask=q_attention_mask,
                token_type_ids=q_token_type_ids,
                position_ids=position_ids,
                head_mask=head_mask,
                inputs_embeds=inputs_embeds,
            )

            q_pooled_output = q_outputs[1]
            q_pooled_output = self.q_dropout(q_pooled_output)

            a_outputs = self.bert(
                a_input_ids,
                attention_mask=a_attention_mask,
                token_type_ids=a_token_type_ids,
                position_ids=position_ids,
                head_mask=head_mask,
                inputs_embeds=inputs_embeds,
            )

            a_pooled_output = a_outputs[1]
            a_pooled_output = self.a_dropout(a_pooled_output)

            emb = [
                emb_layer(cat_features[:, j]) for j, emb_layer in enumerate(self.embeddings)
            ]
            emb = self.emb_drop(torch.cat(emb, 1))

            pooled_output = torch.cat([q_pooled_output, a_pooled_output, num_features, emb], 1)
            logits = self.classifier_final(pooled_output)

            outputs = (logits,) + q_outputs[2:] + a_outputs[2:]  # add hidden states and attention if they are here
            if labels is not None:
                if self.num_labels == 1:
                    #  We are doing regression
                    loss_fct = MSELoss()
                    loss = loss_fct(logits.view(-1), labels.view(-1))
                else:
                    loss_fct = CrossEntropyLoss()
                    loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
                outputs = (loss,) + outputs

            return outputs  # (loss), logits, (hidden_states), (attentions)


def train_model(model, train_loader, optimizer, criterion, scheduler, config):
    
    model.train()
    avg_loss = 0.
    avg_loss_1 = 0.
    avg_loss_2 = 0.
    avg_loss_3 = 0.
    avg_loss_4 = 0.
    avg_loss_5 = 0.
    #tk0 = tqdm(enumerate(train_loader),total =len(train_loader))
    optimizer.zero_grad()
    for idx, batch in enumerate(train_loader):
        if COMBINE_INPUT:
            input_ids, input_masks, input_segments, num_features, cat_features, labels, _ = batch
            input_ids, input_masks, input_segments, num_features, cat_features, labels = input_ids.to(device), input_masks.to(device), input_segments.to(device), num_features.to(device), cat_features.to(device), labels.to(device)            
        
            output_train = model(input_ids = input_ids.long(),
                             labels = None,
                             attention_mask = input_masks,
                             token_type_ids = input_segments,
                             num_features = num_features,
                             cat_features = cat_features,
                            )
        else:
            q_input_ids, q_input_masks, q_input_segments, a_input_ids, a_input_masks, a_input_segments, num_features, cat_features, labels, _ = batch
            q_input_ids, q_input_masks, q_input_segments, a_input_ids, a_input_masks, a_input_segments, num_features, cat_features, labels = q_input_ids.to(device), q_input_masks.to(device), q_input_segments.to(device), a_input_ids.to(device), a_input_masks.to(device), a_input_segments.to(device), num_features.to(device), cat_features.to(device), labels.to(device)

            output_train = model(q_input_ids = q_input_ids.long(),
                             labels = None,
                             q_attention_mask = q_input_masks,
                             q_token_type_ids = q_input_segments,
                             a_input_ids = a_input_ids.long(),
                             a_attention_mask = a_input_masks,
                             a_token_type_ids = a_input_segments,
                             num_features = num_features,
                             cat_features = cat_features,
                            )
        logits = output_train[0] #output preds
        loss = criterion(logits, labels)
        #loss =(config.question_weight*criterion(logits[:,0:21], labels[:,0:21]) + config.answer_weight*criterion(logits[:,21:30], labels[:,21:30]))/config.accum_steps
        loss.backward()
        if (idx + 1) % config.accum_steps == 0:    
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
        
        avg_loss += loss.item() / (len(train_loader)*config.accum_steps)
        if COMBINE_INPUT:
            del input_ids, input_masks, input_segments, labels
        else:
            del q_input_ids, q_input_masks, q_input_segments, a_input_ids, a_input_masks, a_input_segments, labels

    torch.cuda.empty_cache()
    gc.collect()
    return avg_loss, avg_loss_1, avg_loss_2, avg_loss_3, avg_loss_4, avg_loss_5


def val_model(model, criterion, val_loader, val_shape, batch_size=8):

    avg_val_loss = 0.
    model.eval() # eval mode
    
    valid_preds = np.zeros((val_shape, len(target_cols)))
    original = np.zeros((val_shape, len(target_cols)))
    
    #tk0 = tqdm(enumerate(val_loader))
    with torch.no_grad():
        
        for idx, batch in enumerate(val_loader):
            if COMBINE_INPUT:
                input_ids, input_masks, input_segments, num_features, cat_features, labels, _ = batch
                input_ids, input_masks, input_segments, num_features, cat_features, labels = input_ids.to(device), input_masks.to(device), input_segments.to(device), num_features.to(device), cat_features.to(device), labels.to(device)            
            
                output_val = model(input_ids = input_ids.long(),
                               labels = None,
                               attention_mask = input_masks,
                               token_type_ids = input_segments,
                               num_features = num_features,
                               cat_features = cat_features,
                              )
            else:
                q_input_ids, q_input_masks, q_input_segments, a_input_ids, a_input_masks, a_input_segments, num_features, cat_features, labels, _ = batch
                q_input_ids, q_input_masks, q_input_segments, a_input_ids, a_input_masks, a_input_segments, num_features, cat_features, labels = q_input_ids.to(device), q_input_masks.to(device), q_input_segments.to(device), a_input_ids.to(device), a_input_masks.to(device), a_input_segments.to(device), num_features.to(device), cat_features.to(device), labels.to(device)

                output_val = model(q_input_ids = q_input_ids.long(),
                             labels = None,
                             q_attention_mask = q_input_masks,
                             q_token_type_ids = q_input_segments,
                             a_input_ids = a_input_ids.long(),
                             a_attention_mask = a_input_masks,
                             a_token_type_ids = a_input_segments,
                             num_features = num_features,
                             cat_features = cat_features,
                            )
            logits = output_val[0] #output preds
            
            avg_val_loss += criterion(logits, labels).item() / len(val_loader)
            valid_preds[idx*batch_size : (idx+1)*batch_size] = logits.detach().cpu().squeeze().numpy()
            original[idx*batch_size : (idx+1)*batch_size]    = labels.detach().cpu().squeeze().numpy()
        
        score = 0
        preds = torch.sigmoid(torch.tensor(valid_preds)).numpy()
        
        # np.save("preds.npy", preds)
        # np.save("actuals.npy", original)
        
        rho_val = np.mean([spearmanr(original[:, i], preds[:,i]).correlation for i in range(preds.shape[1])])
        print('\r val_spearman-rho: %s' % (str(round(rho_val, 5))), end = 100*' '+'\n')
        
        for i in range(len(target_cols)):
            #print(i, spearmanr(original[:,i], preds[:,i]))
            score += np.nan_to_num(spearmanr(original[:, i], preds[:, i]).correlation)
        
    return avg_val_loss, score/len(target_cols)


def predict_valid_result(model, val_loader, val_length, batch_size=32):

    val_preds = np.zeros((val_length, len(target_cols)))
    original = np.zeros((val_length, len(target_cols)))

    model.eval()
    tk0 = tqdm(enumerate(val_loader))
    for idx, batch in tk0:
        if COMBINE_INPUT:
            input_ids, input_masks, input_segments, num_features, cat_features, labels, _ = batch
            input_ids, input_masks, input_segments, num_features, cat_features, labels = input_ids.to(device), input_masks.to(device), input_segments.to(device), num_features.to(device), cat_features.to(device), labels.to(device)            
            with torch.no_grad():
                outputs = model(input_ids = input_ids.long(),
                            labels = None,
                            attention_mask = input_masks,
                            token_type_ids = input_segments,
                            num_features = num_features,
                            cat_features = cat_features,
                            )
        else:
            q_input_ids, q_input_masks, q_input_segments, a_input_ids, a_input_masks, a_input_segments, num_features, cat_features, labels, _ = batch
            q_input_ids, q_input_masks, q_input_segments, a_input_ids, a_input_masks, a_input_segments, num_features, cat_features, labels = q_input_ids.to(device), q_input_masks.to(device), q_input_segments.to(device), a_input_ids.to(device), a_input_masks.to(device), a_input_segments.to(device), num_features.to(device), cat_features.to(device), labels.to(device)
            with torch.no_grad():
                outputs = model(q_input_ids = q_input_ids.long(),
                             labels = None,
                             q_attention_mask = q_input_masks,
                             q_token_type_ids = q_input_segments,
                             a_input_ids = a_input_ids.long(),
                             a_attention_mask = a_input_masks,
                             a_token_type_ids = a_input_segments,
                             num_features = num_features,
                             cat_features = cat_features,
                            )

        predictions = outputs[0]
        val_preds[idx*batch_size : (idx+1)*batch_size] = predictions.detach().cpu().squeeze().numpy()
        original[idx*batch_size : (idx+1)*batch_size] = labels.detach().cpu().squeeze().numpy()

    output = torch.sigmoid(torch.tensor(val_preds)).numpy()
    return output, original


def predict_result(model, test_loader, test_length, batch_size=32):

    test_preds = np.zeros((test_length, len(target_cols)))

    model.eval()
    tk0 = tqdm(enumerate(test_loader))
    for idx, x_batch in tk0:
        if COMBINE_INPUT:
            with torch.no_grad():
                outputs = model(input_ids = x_batch[0].to(device),
                            labels = None,
                            attention_mask = x_batch[1].to(device),
                            token_type_ids = x_batch[2].to(device),
                            num_features = x_batch[3].to(device),
                            cat_features = x_batch[4].to(device),
                           )
        else:
            with torch.no_grad():
                outputs = model(q_input_ids = x_batch[0].to(device),
                            labels = None,
                            q_attention_mask = x_batch[1].to(device),
                            q_token_type_ids = x_batch[2].to(device),
                            a_input_ids = x_batch[3].to(device),
                            a_attention_mask = x_batch[4].to(device),
                            a_token_type_ids = x_batch[5].to(device),
                            num_features = x_batch[6].to(device),
                            cat_features = x_batch[7].to(device),
                           )
        predictions = outputs[0]
        test_preds[idx*batch_size : (idx+1)*batch_size] = predictions.detach().cpu().squeeze().numpy()

    output = torch.sigmoid(torch.tensor(test_preds)).numpy()
    return output


def add_features(df):
    find = re.compile(r"^[^.]*")
    df['netloc'] = df['url'].apply(lambda x: re.findall(find, urlparse(x).netloc)[0])
    df['qa_same_user_page_flag'] = (df['question_user_page']==df['answer_user_page'])*1
    df['question_title_num_words'] = df['question_title'].str.count('\S+')
    df['question_body_num_words'] = df['question_body'].str.count('\S+')
    df['answer_num_words'] = df['answer'].str.count('\S+')
    df['question_vs_answer_length'] = df['question_body_num_words']/df['answer_num_words']
    df['question_title_num_words'] = np.log1p(df['question_title_num_words'])
    df['question_body_num_words'] = np.log1p(df['question_body_num_words'])
    df['answer_num_words'] = np.log1p(df['answer_num_words'])
    df['question_vs_answer_length'] = np.log1p(df['question_vs_answer_length'])
    return df


def custom_loss(x, y):
    bce_loss = nn.BCEWithLogitsLoss()(x, y)
    return bce_loss


def get_bert_features(model, val_loader, val_length, batch_size=32):

    features = np.zeros((val_length, 768*2))

    model.eval()
    tk0 = tqdm(enumerate(val_loader))
    for idx, batch in tk0:
        if COMBINE_INPUT:
            None
        else:
            q_input_ids, q_input_masks, q_input_segments, a_input_ids, a_input_masks, a_input_segments, num_features, cat_features, labels, _ = batch
            q_input_ids, q_input_masks, q_input_segments, a_input_ids, a_input_masks, a_input_segments, num_features, cat_features, labels = q_input_ids.to(device), q_input_masks.to(device), q_input_segments.to(device), a_input_ids.to(device), a_input_masks.to(device), a_input_segments.to(device), num_features.to(device), cat_features.to(device), labels.to(device)
            with torch.no_grad():
                q_output = model.bert(q_input_ids.long(),
                              attention_mask=q_input_masks,
                              token_type_ids=q_input_segments,
                              position_ids=None,
                              head_mask=None,
                              inputs_embeds=None,
                            )
                a_output = model.bert(a_input_ids.long(),
                              attention_mask=a_input_masks,
                              token_type_ids=a_input_segments,
                              position_ids=None,
                              head_mask=None,
                              inputs_embeds=None,
                            )
        q_feature = q_output[1].detach().cpu().squeeze().numpy()
        a_feature = a_output[1].detach().cpu().squeeze().numpy()
        features[idx*batch_size : (idx+1)*batch_size] = np.hstack([q_feature, a_feature])

    return features


def get_test_bert_features(model, val_loader, val_length, batch_size=32):

    features = np.zeros((val_length, 768*2))

    model.eval()
    tk0 = tqdm(enumerate(val_loader))
    for idx, batch in tk0:
        if COMBINE_INPUT:
            None
        else:
            q_input_ids, q_input_masks, q_input_segments, a_input_ids, a_input_masks, a_input_segments, num_features, cat_features, _ = batch
            q_input_ids, q_input_masks, q_input_segments, a_input_ids, a_input_masks, a_input_segments, num_features, cat_features = q_input_ids.to(device), q_input_masks.to(device), q_input_segments.to(device), a_input_ids.to(device), a_input_masks.to(device), a_input_segments.to(device), num_features.to(device), cat_features.to(device)
            with torch.no_grad():
                q_output = model.bert(q_input_ids.long(),
                              attention_mask=q_input_masks,
                              token_type_ids=q_input_segments,
                              position_ids=None,
                              head_mask=None,
                              inputs_embeds=None,
                            )
                a_output = model.bert(a_input_ids.long(),
                              attention_mask=a_input_masks,
                              token_type_ids=a_input_segments,
                              position_ids=None,
                              head_mask=None,
                              inputs_embeds=None,
                            )
        q_feature = q_output[1].detach().cpu().squeeze().numpy()
        a_feature = a_output[1].detach().cpu().squeeze().numpy()
        features[idx*batch_size : (idx+1)*batch_size] = np.hstack([q_feature, a_feature])

    return features


#===========================================================
# main
#===========================================================
if True:
    
    with timer('Data Loading'):
        train = pd.read_csv(f"{ROOT}train.csv").fillna("none")
        y_train = train[target_cols].values
        if config.test:
            test = pd.read_csv(f"{ROOT}test.csv").fillna("none")
            submission = pd.read_csv(f"{ROOT}sample_submission.csv")
    
    with timer('Num features'):
        train = add_features(train)
        if config.test:
            test = add_features(test)
        num_features = ['question_title_num_words', 'question_body_num_words', 'answer_num_words', 'question_vs_answer_length']
        train_num = train[num_features].values
        if config.test:
            test_num = test[num_features].values
                
    with timer('Cat features'):
        cat_features = ['netloc', 'category', 'qa_same_user_page_flag']
        ce_oe = ce.OrdinalEncoder(cols=cat_features, handle_unknown='return_nan')
        ce_oe.fit(train[cat_features])
        train_cat_df = ce_oe.transform(train[cat_features])
        test_cat_df = ce_oe.transform(test[cat_features]).fillna(0).astype(int)
        for c in cat_features:
            train[c] = train_cat_df[c]
            test[c] = test[c]
        train_cat = train_cat_df.values
        test_cat = test_cat_df.values
        cat_dims = []
        for col in cat_features:
            dim = train[col].nunique()
            cat_dims.append((dim+1, dim//2+1))
        print(cat_dims)

[Data Loading] start
[Data Loading] done in 0 s
[Num features] start
[Num features] done in 1 s
[Cat features] start
[Cat features] done in 0 s


[(60, 30), (6, 3), (3, 2)]


In [2]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
import pickle


def min_max(x, axis=None):
    _min = x.min(axis=axis, keepdims=True)
    _max = x.max(axis=axis, keepdims=True)
    result = (x-_min)/(_max-_min)
    return result


def custom_eval(y_pred, data):
    y_true = data.get_label()
    y_pred = y_pred.reshape(-1)
    #N_LABELS = 9
    #y_pred = y_pred.reshape(N_LABELS, len(y_preds) // N_LABELS)
    score = spearmanr(y_true, y_pred).correlation
    return 'spearmanr', score, True # name, result, is_higher_better

def run_single_lightgbm(param, train_df, test_df, folds, features, target, fold_num=0, categorical=[], target_col=''):
    
    trn_idx = folds[folds.fold != fold_num].index
    val_idx = folds[folds.fold == fold_num].index
    logger.info(f'len(trn_idx) : {len(trn_idx)}')
    logger.info(f'len(val_idx) : {len(val_idx)}')
    
    if categorical == []:
        trn_data = lgb.Dataset(train_df.iloc[trn_idx][features],
                               label=target.iloc[trn_idx])
        val_data = lgb.Dataset(train_df.iloc[val_idx][features],
                               label=target.iloc[val_idx])
    else:
        trn_data = lgb.Dataset(train_df.iloc[trn_idx][features],
                               label=target.iloc[trn_idx],
                               categorical_feature=categorical)
        val_data = lgb.Dataset(train_df.iloc[val_idx][features],
                               label=target.iloc[val_idx],
                               categorical_feature=categorical)

    oof = np.zeros(len(train_df))
    predictions = np.zeros(len(test_df))

    num_round = 10000

    clf = lgb.train(param,
                    trn_data,
                    num_round,
                    valid_sets=[trn_data, val_data],
                    verbose_eval=1000,
                    early_stopping_rounds=100,
                    #feval=custom_eval,
                   )
    
    logger.info(f'Dumping model with pickle... {target_col}_lightgbm_fold{fold_num}.pkl')
    with open(f'{target_col}_lightgbm_fold{fold_num}.pkl', 'wb') as fout:
        pickle.dump(clf, fout)

    oof[val_idx] = clf.predict(train_df.iloc[val_idx][features], num_iteration=clf.best_iteration)
    oof[val_idx] = min_max(oof[val_idx], axis=None)

    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = features
    fold_importance_df["importance"] = clf.feature_importance(importance_type='gain')
    fold_importance_df["fold"] = fold_num

    #predictions += clf.predict(test_df[features], num_iteration=clf.best_iteration)
    
    # spearmanr
    rho_val = spearmanr(target[val_idx], oof[val_idx]).correlation
    logger.info(f'spearmanr: {rho_val}')
    
    return oof, predictions, fold_importance_df

In [3]:
import warnings
warnings.filterwarnings("ignore")

nn_oof = pd.read_csv(MODEL_DIR+'oof.csv')
folds = pd.read_csv(MODEL_DIR+'folds.csv')
output = pd.DataFrame()
output[ID] = folds[ID]

for i in range(len(target_cols)):

    target_col = target_cols[i]
    target = train[target_col]
    logger.info('============================================================================')
    logger.info(f'target: {target_col}  nn CV spearmanr: {spearmanr(target, nn_oof[target_col]).correlation}')
    logger.info('============================================================================')

    oof = np.zeros(len(train))

    for fold in range(NUM_FOLDS):

        train_cols = ['category', 'netloc', 'qa_same_user_page_flag', 'question_title_num_words',
                      'question_body_num_words', 'answer_num_words', 'question_vs_answer_length']
        bert_features = pd.read_csv(f'../input/lgb-with-bert-features/train_bert_features_{fold}.csv')
        df = pd.concat([train[train_cols], bert_features], axis=1)
        num_features = [c for c in df.columns if df.dtypes[c] != 'object']
        cat_features = ['netloc', 'category', 'qa_same_user_page_flag']
        features = num_features + cat_features
        drop_features = ['qa_id']
        features = [c for c in features if c not in drop_features]

        lgb_param = {
                    #'objective': 'regression',
                    'metric': 'rmse',
                    'objective': 'xentropy',
                    'boosting_type': 'gbdt',
                    'learning_rate': 0.01,
                    'data_random_seed': 42,
                    'num_leaves': 25,
                    'subsample': 0.7,
                    'subsample_freq': 2,
                    'max_depth': 6,
                    'reg_alpha': 0.1,
                    'colsample_bytree': 0.7,
                    'min_split_gain': 0.5,
                    'reg_lambda': 0.1,
                    'min_data_in_leaf': 100,
                    'verbosity': -1,
                }

        _oof, _, _ = run_single_lightgbm(lgb_param, df, df, folds, features, target, fold_num=fold, 
                                         categorical=cat_features, target_col=target_col)
        oof += _oof

    # spearmanr
    rho_val = spearmanr(target, oof).correlation
    logger.info('============================================================================')
    logger.info(f'target: {target_col}  LGB CV spearmanr: {rho_val}')
    logger.info('============================================================================')
    
    output[target_col] = oof

target: question_asker_intent_understanding  nn CV spearmanr: 0.35272149968282696
len(trn_idx) : 4866
len(val_idx) : 1213


Training until validation scores don't improve for 100 rounds


Dumping model with pickle... question_asker_intent_understanding_lightgbm_fold0.pkl


Early stopping, best iteration is:
[188]	training's rmse: 0.109672	valid_1's rmse: 0.124939


spearmanr: 0.3893010387900006
len(trn_idx) : 4866
len(val_idx) : 1213


...

spearmanr: 0.20598549179642242
len(trn_idx) : 4856
len(val_idx) : 1223


Training until validation scores don't improve for 100 rounds


Dumping model with pickle... answer_well_written_lightgbm_fold4.pkl


Early stopping, best iteration is:
[102]	training's rmse: 0.0910428	valid_1's rmse: 0.0978224


spearmanr: 0.2147512686540319
target: answer_well_written  LGB CV spearmanr: 0.19364566242465361


In [4]:
output.head()

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,0,0.919385,0.763212,0.133850,0.532502,0.769519,0.452999,0.439386,0.548741,0.087481,...,0.938038,0.975896,0.610017,0.987376,0.969892,0.876631,0.050595,0.199820,0.807416,0.835579
1,1,0.802717,0.870660,0.030838,0.425130,0.975821,0.802414,0.606284,0.508436,0.286987,...,0.846635,0.606320,0.396153,0.624009,0.741587,0.625852,0.001418,0.025773,0.803415,0.792329
2,2,0.801068,0.114099,0.024304,0.745167,0.804123,0.911030,0.375687,0.254366,0.468644,...,0.522497,0.958741,0.597884,0.972841,0.975081,0.887242,0.696899,0.296545,0.706149,0.825503
3,3,0.829096,0.727061,0.058715,0.814212,0.765017,0.856277,0.436492,0.467468,0.431506,...,0.879282,0.695098,0.702164,0.706668,0.807679,0.673661,0.120646,0.313440,0.689994,0.580218
4,5,0.833635,0.666730,0.025321,0.660665,0.754761,0.829150,0.493166,0.387803,0.084927,...,0.855202,0.596161,0.229608,0.551253,0.482131,0.503570,0.776684,0.224922,0.074021,0.211178


In [5]:
output.to_csv('lgb_oof.csv', index=False)